# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Basic text operation with Gemini


## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

## Authentication to access to the GCP & Google drive

In [ ]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # Mount to the google drive to access the .ipynb files in the repository.

    # from google.colab import drive
    # drive.mount('/content/drive')

Mounted at /content/drive


## Set the environment on GCP Project

In [ ]:
# Set the environment on GCP Project

PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL_NAME="gemini-1.5-flash"

## Vertex AI initialization


In [ ]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)


# Initialize the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION) # https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

system_instructions = [
 "Use Korean when responding",
 "Use table format to categorize the facts"
]

# Access to the generative model.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

## Function to get the response

In [ ]:
def generate(query:str):
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    # Print the model response
    print(f'\nUsage metadata:\n{responses.to_dict().get("usage_metadata")}')
    print(f"\nFinish reason:\n{responses.candidates[0].finish_reason}")
    print(f"\nSafety settings:\n{responses.candidates[0].safety_ratings}")

    return responses.text

## Run example

In [ ]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
You are a Generative AI specialist to help people understand the concept of AI.
Explain what's the pros and cons of using AI in the following instructions.
1. Define what's the Generative AI.
2. What are differences between typical machine learning and Generative AI.
3. What are the pros and cons of using Generative AI.
4. Explain use cases of Generative AI in the financial industry.

"""

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))



Token Count : total_tokens: 92
total_billable_characters: 321


Usage metadata:
{'prompt_token_count': 103, 'candidates_token_count': 608, 'total_token_count': 711}

Finish reason:
1

Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.08251953125
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.08740234375
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.055908203125
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.140625
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.126953125
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.11572265625
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.046630859375
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.07177734375
]
Time : 5.225049363999915 seconds




## 생성형 AI: 개념, 장단점 및 금융 분야 활용 사례

### 1. 생성형 AI 정의

| 용어 | 설명 |
|---|---|
| 생성형 AI | 새로운 콘텐츠를 생성하는 데 중점을 둔 인공지능의 한 유형입니다. 텍스트, 이미지, 오디오, 비디오, 코드와 같은 다양한 형태의 콘텐츠를 생성할 수 있습니다. |

### 2. 일반적인 머신 러닝과 생성형 AI의 차이점

|  특징  | 일반적인 머신 러닝 | 생성형 AI |
|---|---|---|
| 목표 |  기존 데이터를 기반으로 예측 또는 분류 수행 | 새로운 콘텐츠 생성 |
| 학습 방식 |  지도 학습, 비지도 학습, 강화 학습 |  지도 학습, 비지도 학습, 강화 학습 (일부) |
| 예시 |   이미지 분류, 텍스트 감정 분석 |  텍스트 생성, 이미지 생성, 음악 작곡 |

### 3. 생성형 AI 사용의 장단점

|  장점  | 단점 |
|---|---|
|  새로운 콘텐츠 생성 |  편견 가능성 |
|  창의성 및 효율성 향상 |  가짜 콘텐츠 생성 가능성 |
|  개인 맞춤형 콘텐츠 제공 |  저작권 문제 발생 가능성 |
|  자동화된 작업 처리 |  데이터 프라이버시 문제 |

### 4. 금융 분야에서의 생성형 AI 활용 사례

|  사례  | 설명 |
|---|---|
|  금융 보고서 작성 |  자동화된 보고서 생성 및 분석 |
|  금융 사기 탐지 |  사기 패턴 학습 및 탐지 |
|  고객 맞춤형 금융 상품 추천 |  고객 선호도 분석 및 맞춤형 상품 추천 |
|  투자 전략 개발 |  시장 데이터 분석 및 최적의 투자 전략 생성 |
|  챗봇 및 가상 비서 |  고객 문의 자동 응답 및 금융 서비스 제공 |

**결론**

생성형 AI는 금융 분야를 비롯한 다양한 분야에서 혁신적인 변화를 가져올 잠재력을 가지고 있습니다. 그러나 편견, 가짜 콘텐츠 생성, 저작권, 데이터 프라이버시 등의 문제를 해결하기 위한 노력이 필요합니다. 생성형 AI를 윤리적이고 책임감 있게 활용하여 사회적 이익을 창출할 수 있도록 지속적인 연구 개발 및 사회적 논의가 필요합니다.
